In [1]:
import pandas as pd
import csv
import numpy as np
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split

In [2]:
column_types = {
    'isAdult': float,
    'startYear': float,
    'endYear': float,
    'runtimeMinutes': float,
    'tconst': str,
    'titleType': str,
    'primaryTitle': str,
    'originalTitle': str,
    'genres': str
}

titles_df = pd.read_csv("data.tsv", 
                        dtype=column_types,
                        na_values=r'\N',
                        sep="\t",
                        quoting=csv.QUOTE_NONE)

titles_df = titles_df.dropna(subset=['isAdult','runtimeMinutes','startYear'])

MemoryError: Unable to allocate 318. MiB for an array with shape (4, 10408721) and data type float64

In [ ]:
print(titles_df.shape)
titles_df.head()

In [ ]:
titles_df['titleType'].unique()

In [ ]:
tv_types = [
    'tvMovie',
    'tvSeries',
    'tvEpisode',
    'tvShort',
    'tvMiniSeries',
    'tvSpecial'
]
titles_df = titles_df.loc[titles_df['titleType'].isin(tv_types)]

In [ ]:
titles_df = titles_df.loc[titles_df['primaryTitle'] == titles_df['originalTitle']]
titles_df

In [7]:
def principal_component_analysis(dataframe, key_list):
    new_dataframe = dataframe[key_list]

    # Compute the mean and std of the data
    mean = np.mean(new_dataframe, axis=0)
    std = np.std(new_dataframe, axis=0)
    # Standardize the data by subtracting the mean and dividing by std
    standardized_data = (new_dataframe - mean) / std
    
    # Compute the covariance matrix
    cov_matrix = np.cov(standardized_data, rowvar=False)

    # Get the eigenvalues and eigenvectors
    eigen_values, eigen_vectors = np.linalg.eig(cov_matrix)

    # Sort the eigenvectors by decreasing eigenvalues
    sorted_index = np.argsort(eigen_values)[::-1]
    sorted_eigenvalue = eigen_values[sorted_index]
    sorted_eigenvectors = eigen_vectors[:, sorted_index]

    # Transform the data
    return np.dot(standardized_data, sorted_eigenvectors)

In [8]:
foo = principal_component_analysis(titles_df,["isAdult",'runtimeMinutes', "startYear"])
foo

array([[-2.24930501, -1.92286087, -2.18237185],
       [-2.85453809,  0.03628494, -2.51660433],
       [-2.07057413, -1.70349914, -1.98260552],
       ...,
       [ 0.44219416,  0.32168503,  0.69178961],
       [ 0.48927506, -0.45459475,  0.63876866],
       [ 0.49631782, -0.47591732,  0.64284475]])

In [9]:
class GaussianNaiveBayes: 
    def __init__(self, x, y, log=False) :
        self.data = np.concatenate((np.array(x), np.array(y).reshape(-1, 1)), axis=1)
        self.n_features = x.shape[1]
        self.classes = set(y)
        self.log = log
    
    def fit(self) :
        self.mean, self.std, self.len = np.zeros(len(self.classes)),np.zeros(len(self.classes)),np.zeros(len(self.classes))

        for i in self.classes : 
            d = self.data[self.data[:,-1]==i]
            self.mean[i] = np.mean(d, axis=0)[:-1]
            self.std[i] = np.std(d, axis=0)[:-1],
            self.len[i] = len(d)
    
    def get_probability(self, inp, mean, std) :
        res = (1 / (np.sqrt(2 * np.pi) * std)) * np.exp(-(((inp-mean)**2)/(2*(std**2))))
        if self.log : 
            return np.log(1+res)
        return res
    
    def predict(self, test_set) :
        results = []
        t = 0
        for inp in test_set : 
            pred_class, pred_prob = -1 , 0
            for i in self.classes: 
                probs = self.get_probability(inp, self.mean[i], self.std[i])
                class_prob = np.prod(probs)
                if class_prob > pred_prob : 
                    pred_class, pred_prob = i , class_prob
            results.append(pred_class)
            t += 1
        return results


In [11]:
X_train, X_test, y_train, y_test = train_test_split(titles_df[["primaryTitle","startYear", "runtimeMinutes"]], titles_df["isAdult"], test_size=0.9, random_state=0)

gnb1 = GaussianNaiveBayes(X_train,y_train)
gnb1.fit()
y_pred1 = gnb1.predict(X_test)
print("Result 1:" ,y_pred1 )

Init


MemoryError: 

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(titles_df[["primaryTitle","startYear", "runtimeMinutes"]], titles_df["isAdult"], test_size=0.5, random_state=0)

gnb2 = GaussianNB()
y_pred2 = gnb2.fit(X_train, y_train).predict(X_test)
y_pred2
print("Result 2:" , y_pred2)